In [27]:
import pandas as pd
import fasttext
from gensim.utils import tokenize
import random
import numpy as np


In [8]:
clotho = pd.read_csv("raw_data/clotho_captions_evaluation.csv")

In [ ]:
# Note that the embedding dimension is 300 
fasttext_model =  fasttext.load_model('pre-trained_fasttext/crawl-300d-2M-subword.bin')

In [45]:
def Combine_caption_columns_into_one_column(df,fasttext_model):
    """Converts a df with file_name and multiple caption columns into 1 column

    file_name          caption_1        caption_2       caption_3  caption_4 
    young artists.wav  male and women   youth aspiring  ..         ..
    
    ↓

    file_name          caption
    young artists.wav  male and women
    young artists.wav  youth aspiring
    young artists.wav  ...
    
    Input:df 
    Output:converted df 
"""
    temp_lst = []
    counter = 0
    for k,v in df.iterrows():
        file_name = v[0]
        for i in range(1,len(v)):

            tokenized_words = list(tokenize(v[i]))

            # if counter in (0,1,2,3):
            #     print(len(tokenized_words),tokenized_words)
            #     counter += 1

            # Exchangable word-to-vect model
            word_vecs = [fasttext_model.get_word_vector(j) for j in tokenized_words]
            # To be changed, if bad performanceword_vecs = [fasttext_model.get_word_vector(j) for j in tokenized_words]
            avg_word_vecs = np.divide(np.sum(word_vecs, axis=0),len(tokenized_words))
            temp_lst.append([file_name, avg_word_vecs])
    # Sorted for easier searching
    df = pd.DataFrame(temp_lst,columns=['file_name','caption']).sort_values(by=['file_name']).reset_index(drop=True)
    return df

In [46]:
input_df = Combine_caption_columns_into_one_column(clotho,fasttext_model)
input_df

,file_name,caption
0,01 A pug struggles to breathe 1_14_2008.wav,"[-0.0010263734, 0.032575328, 0.07291455, 0.003..."
1,01 A pug struggles to breathe 1_14_2008.wav,"[-0.0027864054, 0.026300091, 0.08160074, -0.00..."
2,01 A pug struggles to breathe 1_14_2008.wav,"[0.00487091, 0.009667663, 0.042670406, 0.00672..."
3,01 A pug struggles to breathe 1_14_2008.wav,"[0.0010986233, -0.017237674, 0.081616476, 0.01..."
4,01 A pug struggles to breathe 1_14_2008.wav,"[0.010673653, 0.01178974, 0.045919742, 0.01241..."
...,...,...
5220,young artists.wav,"[0.013803961, -0.04192563, 0.06460683, 0.00279..."
5221,young artists.wav,"[0.00072285323, 0.0025999807, 0.04834584, 0.00..."
5222,young artists.wav,"[0.0021603287, -0.011772836, 0.051467545, 0.00..."
5223,young artists.wav,"[0.0020250888, -0.02334438, 0.063647024, 0.005..."


In [47]:
# Sanity Check
print(len(input_df['caption'][4]))
print(len(input_df['caption'][10]))


300
300
